# Dataset

In [2]:
import pandas as pd

import os

from module import toSunpyMap

from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.coordinates import HeliographicStonyhurst

import matplotlib.pyplot as plt


In [3]:
def convert_to_lon_lat(location):
    # Latitude
    lat_sign = 1 if location[0] == 'N' else -1
    latitude = lat_sign * int(location[1:3])
    
    # Longitude
    lon_sign = -1 if location[3] == 'E' else 1
    longitude = lon_sign * int(location[4:])
    
    return longitude, latitude

In [4]:
df = pd.read_csv('FlareCatalogue.csv')

In [5]:
event = df.iloc[16]
event['start']

'2023/07/29 16:11'

In [6]:
lon, lat = convert_to_lon_lat(event['AR location'])
lon, lat

(8, -11)

In [10]:
import os
from datetime import datetime, timedelta
import re

def getTimeSeriesForEvent(event)

    # Define the event date and time
    event_datetime_str = event['start']
    event_datetime = datetime.strptime(event_datetime_str, '%Y/%m/%d %H:%M')

    directory = 'Y:/data/CLIMSO/'+str(event_datetime.year)+'/data_calibrated/'+event_datetime.strftime('%Y-%m-%d')+'/'

    # List all files in the directory that match the pattern
    files = [file for file in os.listdir(directory) if file.startswith('imoa_03933_l2_')]

    # Extract datetime from filename using regex
    def extract_datetime_from_filename(filename):
        match = re.search(r'_(\d{8}_\d{6}\d{2})_', filename)  # Adjusted regex to match milliseconds
        if match:
            datetime_str = match.group(1)
            try:
                return datetime.strptime(datetime_str, '%Y%m%d_%H%M%S%f')
            except ValueError as e:
                print(f"Error parsing datetime string: {datetime_str}, error: {e}")
        return None

    # Create a list of (filename, datetime) tuples
    file_datetimes = [(file, extract_datetime_from_filename(file)) for file in files]

    # Filter out any files that did not match the pattern or had parsing issues
    file_datetimes = [fd for fd in file_datetimes if fd[1] is not None]

    # Sort files by datetime
    file_datetimes.sort(key=lambda x: x[1])

    # Function to find the closest file to a given time
    def find_closest_file(target_time, files):
        closest_file = None
        min_diff = timedelta.max
        for file, file_time in files:
            diff = abs(file_time - target_time)
            if diff < min_diff:
                min_diff = diff
                closest_file = (file, file_time)
        return closest_file

    # Find the files closest to each 1-hour interval before the event datetime
    closest_files = []
    current_time = event_datetime

    for i in range(7):
        current_time -= timedelta(hours=1)
        closest_file = find_closest_file(current_time, file_datetimes)
        if closest_file:
            closest_files.append(closest_file)
            # Remove the found file to avoid duplicate selections
            file_datetimes.remove(closest_file)

    # Extract filenames from the tuples
    closest_filenames = [fd[0] for fd in closest_files]

    # Output the closest filenames
    return closest_filenames


['imoa_03933_l2_20230729_15113277_emi1.fts.bz2',
 'imoa_03933_l2_20230729_14104102_emi1.fts.bz2',
 'imoa_03933_l2_20230729_13103723_emi1.fts.bz2',
 'imoa_03933_l2_20230729_12095874_emi1.fts.bz2',
 'imoa_03933_l2_20230729_11493017_emi1.fts.bz2',
 'imoa_03933_l2_20230729_09483768_emi1.fts.bz2',
 'imoa_03933_l2_20230729_09155787_emi1.fts.bz2']